# Tools

This notebook contains all the tools that will be used by the agent.

In [1]:
import requests
import sqlite3
import pandas as pd

from transformers import Tool
from smolagents import tool

## Creating simple tool

A Python function must be annotated with `@tool`. It should also have a docstring describing what does the function do, what does it return and the description of its parameters.

### City to location 

The following function look up at latitude and longitude of a city.

In [5]:
# TODO: Load CSV file containing latitude, longitude and altitude of cities from the file data/cities_latlng.csv
# TODO: Use df as the variable name
# https://github.com/bahar/WorldCityLocations/tree/master

df = pd.read_csv('./data/cities_latlng.csv', sep=";")
df.columns = [ 'id', 'country', 'city', 'latitude', 'longitude', 'altitude']
print(df.head())

df.drop('country', axis=1, inplace=True)

   id      country            city   latitude  longitude  altitude
0   2  Afghanistan        Kandahar  31.610000  65.699997    1015.0
1   3  Afghanistan  Mazar-e Sharif  36.706944  67.112221     369.0
2   4  Afghanistan           Herat  34.340000  62.189999     927.0
3   5  Afghanistan       Jalalabad  34.420000  70.449997     573.0
4   6  Afghanistan          Konduz  36.720000  68.860001     394.0


In [6]:
# TODO: Explore the loaded dataframe
df[df['city'] == 'Tokyo']


,id,city,latitude,longitude,altitude
5481,5483,Tokyo,35.689527,139.691681,40.0


In [13]:
# TODO: Add tool description

@tool
def get_latlng(city: str) -> dict:
   """ 
   Return the latitude, longitude and altitude of a city in a dictionary with the following 4 keys: city, latitude, longitude, altitude

   Args:
      city: the name of the city that you want the latitude, longitude and altitude 
   """
   r = df.query(f"city.str.lower() == '{city.lower()}'")
   if len(r) <= 0:
      return None
   return { 'city': city, 'latitude': r.iloc[0]['latitude'], 'longitude': r.iloc[0]['longitude'], 'altitude': r.iloc[0]['altitude'] }

In [10]:
# TODO: Test get_latlng method
# TODO: perform a case insensitive search
get_latlng('tokyo')


{'city': 'tokyo',
 'latitude': 35.6895266,
 'longitude': 139.6916809,
 'altitude': 40.0}

### Temperature at latitude and longitude

The following function lookup the weather at the given latitude and longtude.

In [19]:
# TODO: Add tool description

@tool
def get_temperature(latitude: float, longitude: float) -> dict:
   """ 
   Return the temperature of a location given by the latitude and longitude in a dictionary with the following 2 keys: temperature_unit, temperature

   Args:
      latitude: latitude of a city, a number
      longitude: longitude of a city, a number
   """
   url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m"
   resp = requests.get(url)
   j = resp.json()
   if resp.status_code >= 400:
      raise Exception(j['reason'])
   temperature = j['current']['temperature_2m']
   units = j['current_units']['temperature_2m']
   return { "temperature_unit": units, "temperature": temperature }

In [20]:
# TODO: Test get_temperature method

# case insensitive search
pos = get_latlng('mumbai')
temperature = get_temperature(pos['latitude'], pos['longitude'])
print(temperature)

{'temperature_unit': '°C', 'temperature': 22.0}


### Query relational database

The following function queries a relational database (SQLite) view called `album_track`. The table's schema is as follows:
| Field name  | Type          |
|-------------|---------------| 
| AlbumId     | integer       |
| Title       | nvarchar(160) |
| track_name  | nvarchar(200) |
| artist_name | nvarchar(120) |
| duration    | integer       |
| composer    | nvarchar(220) |


In [21]:
# TODO: Add tool description

def query_album_track(query: str) -> str:
   database = "data/chinook_sqlite.sqlite" 
   conn = sqlite3.connect(database)
   try:
      cursor = conn.cursor() 
      rows = cursor.execute(query)
      return rows.fetchall()
   finally:
      conn.close()

In [24]:
# TODO: Test the query_album_track function
query_album_track('select distinct artist_name from album_track')

[('AC/DC',),
 ('Accept',),
 ('Aerosmith',),
 ('Alanis Morissette',),
 ('Alice In Chains',),
 ('Antônio Carlos Jobim',),
 ('Apocalyptica',),
 ('Audioslave',),
 ('BackBeat',),
 ('Billy Cobham',),
 ('Black Label Society',),
 ('Black Sabbath',),
 ('Body Count',),
 ('Bruce Dickinson',),
 ('Buddy Guy',),
 ('Caetano Veloso',),
 ('Chico Buarque',),
 ('Chico Science & Nação Zumbi',),
 ('Cidade Negra',),
 ('Cláudio Zoli',),
 ('Various Artists',),
 ('Led Zeppelin',),
 ('Frank Zappa & Captain Beefheart',),
 ('Marcos Valle',),
 ('Metallica',),
 ('Queen',),
 ('Kiss',),
 ('Spyro Gyra',),
 ('Green Day',),
 ('David Coverdale',),
 ('Gonzaguinha',),
 ('Os Mutantes',),
 ('Deep Purple',),
 ('Santana',),
 ('Ed Motta',),
 ('Miles Davis',),
 ('Gene Krupa',),
 ('Toquinho & Vinícius',),
 ('Creedence Clearwater Revival',),
 ('Cássia Eller',),
 ('Def Leppard',),
 ('Dennis Chambers',),
 ('Djavan',),
 ('Elis Regina',),
 ('Eric Clapton',),
 ('Faith No More',),
 ('Falamansa',),
 ('Foo Fighters',),
 ('Frank Sinatra',)

### Tools with states

The following isn an example of a more complex tool that requires initialisation

In [25]:
class SQLiteTool(Tool):

   # Required metadata for the agent
   name = "chinook_sqlite"

   description = (""" 
      Perform SQL queries on the album_track table. Returns the result as an array of dictionary.
      The table name is album_track and it has the following columns:
            AlbumId: integer
            Title: nvarchar(160)
            track_name: nvarchar(200)
            artist_name: nvarchar(120)
            duration: integer 
            composer: nvarchar(220)
      The duration column is in milliseconds
   """)

   # JSON schema
   inputs = {
      "query": {
         "type": "string",
         "description": "A valid SQL query"
      }      
   }

   output_type = "string"

   def __init__(self, db_file):
      self.db_file = db_file 

   # The tool method
   def __call__(self, query: str) -> list:
      conn = sqlite3.connect(self.db_file)
      try:
         cursor = conn.cursor() 
         rows = cursor.execute(query)
         return rows.fetchall()
      finally:
         conn.close()